In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  #显示中文
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
plt.rcParams['figure.dpi'] = 300
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # 显示所有列
from tqdm.notebook import tqdm
import geopandas as gpd
import transbigdata as tbd
import geoplot as gplt

In [2]:
# 读取四环数据，用于裁切，本文的数据是四环内的数据
sihuan = gpd.read_file('E:/Data/成都数据/成都道路_好用/成都四环/四环.shp')
sihuan = sihuan.to_crs(epsg=4326)

In [3]:
real_trip = pd.read_csv('E:/Data/mobike201803/trip_data_3_7.csv')
real_trip['startlng'] = tbd.gcj02towgs84(real_trip['startlng'], real_trip['startlat'])[0]
real_trip['startlat'] = tbd.gcj02towgs84(real_trip['startlng'], real_trip['startlat'])[1]
real_trip = gpd.GeoDataFrame(real_trip, geometry=gpd.points_from_xy(real_trip.startlng, real_trip.startlat))
real_trip_in_sihuan = real_trip.clip(mask=sihuan)

### 将trip匹配到网格上

In [10]:
# 时间格式转换
real_trip_in_sihuan['starttime'] = pd.to_datetime(real_trip_in_sihuan['starttime']) 
real_trip_in_sihuan['endtime'] = pd.to_datetime(real_trip_in_sihuan['endtime'])
real_trip_in_sihuan['hour'] = real_trip_in_sihuan['starttime'].dt.hour

In [15]:
trip_in_sihuan

,bikeid,start_time,start_lng,start_lat,end_time,end_lng,end_lat,time_difference,distance,geometry
406714,8640156819,2018-03-07 13:24:00,103.951133,30.611697,2018-03-07 13:28:00,103.973865,30.621637,4.0,3372.136699,POINT (103.95113 30.61170)
631828,8642534618,2018-03-07 12:24:00,103.948405,30.612442,2018-03-07 13:10:00,103.941945,30.593038,46.0,2671.093343,POINT (103.94841 30.61244)
595618,8642519455,2018-03-07 07:18:00,103.950384,30.612851,2018-03-07 07:28:00,103.968122,30.606207,10.0,1922.827863,POINT (103.95038 30.61285)
594286,8642519147,2018-03-07 18:48:00,103.950612,30.614573,2018-03-07 19:10:00,103.969283,30.615614,22.0,1996.658444,POINT (103.95061 30.61457)
593973,8642519084,2018-03-07 18:44:00,103.957181,30.605080,2018-03-07 19:40:00,103.977148,30.597530,56.0,2235.749276,POINT (103.95718 30.60508)
...,...,...,...,...,...,...,...,...,...,...
15081,280073597,2018-03-07 09:32:00,104.086212,30.760892,2018-03-07 09:50:00,104.093166,30.771761,18.0,1888.804401,POINT (104.08621 30.76089)
216671,286673662,2018-03-07 07:20:00,104.086248,30.760903,2018-03-07 07:32:00,104.075276,30.756928,12.0,1470.586440,POINT (104.08625 30.76090)
546541,8640260628,2018-03-07 07:16:00,104.086342,30.760939,2018-03-07 07:38:00,104.055885,30.739830,22.0,5233.055428,POINT (104.08634 30.76094)
499012,8640246192,2018-03-07 09:46:00,104.086436,30.765354,2018-03-07 10:04:00,104.112960,30.763140,18.0,2305.117522,POINT (104.08644 30.76535)


In [22]:
grid_m_list = np.append((np.linspace(100,900,9)),\
          np.linspace(1000,5000,9)) # 生成网格大小列表

for samp_rate in tqdm(range(2,21)):
    # 读取trip数据
    trip = pd.read_csv('E:/Data/mobike201803/location_to_trip_data_new_gcj2wgs/loc2trip_%d.csv'%samp_rate,encoding='utf-8-sig')
    trip = gpd.GeoDataFrame(trip, geometry=gpd.points_from_xy(trip['start_lng'], trip['start_lat']), crs='epsg:4326')
    trip_in_sihuan = trip.clip(mask=sihuan)
    trip_in_sihuan['start_time'] = pd.to_datetime(trip_in_sihuan['start_time'])
    trip_in_sihuan['end_time'] = pd.to_datetime(trip_in_sihuan['end_time'])
    trip_in_sihuan['hour'] = trip_in_sihuan['start_time'].dt.hour

    # 将trip_in_sihuan连接到grid上
    for grid_m in grid_m_list:
        grid_m = int(grid_m)
        print('采样率： %dmin  and'%samp_rate, '  网格大小：', grid_m, '米', end='----')

        # 读取网格
        grid = pd.read_csv('E:/Data/成都数据/rect_road_poi/rect%d.csv'%grid_m,encoding='utf-8-sig')
        from shapely import wkt
        grid['geometry'] = grid['geometry'].apply(wkt.loads)
        grid = gpd.GeoDataFrame(grid, geometry='geometry', crs='epsg:4326')

        grid_trip_num = pd.DataFrame()
        # 分时间段进行匹配
        for hour in range(24):
            trip_in_sihuan_hour = trip_in_sihuan[trip_in_sihuan['hour']==hour]
            real_trip_in_sihuan_hour = real_trip_in_sihuan[real_trip_in_sihuan['hour']==hour]

            # 将trip_in_sihuan连接到当前grid上，对应generate_trip_num 
            dfsjoin = gpd.sjoin(grid, trip_in_sihuan_hour)  # Spatial join Points to polygons
            # 根据poly_id分组，计算每个网格内的POI数量
            dfpivot = pd.pivot_table(dfsjoin, index='poly_id', aggfunc={'bikeid': len})
            # 重命名列名
            dfpivot.rename(columns={'bikeid': 'generate_trip_num'}, inplace=True)
            grid_trip_num_hour = grid.merge(dfpivot, how='left', on='poly_id') 

            # 将real_trip_in_sihuan连接到当前grid上 ，对应real_trip_num  
            dfsjoin = gpd.sjoin(grid, real_trip_in_sihuan_hour)  # Spatial join Points to polygons
            # 根据poly_id分组，计算每个网格内的POI数量
            dfpivot = pd.pivot_table(dfsjoin, index='poly_id', aggfunc={'bikeid': len})
            # 重命名列名
            dfpivot.rename(columns={'bikeid': 'real_trip_num'}, inplace=True)
            grid_trip_num_hour = grid_trip_num_hour.merge(dfpivot, how='left', on='poly_id') 
            # 添加hour列
            grid_trip_num_hour['hour'] = hour
            
            
            grid_trip_num = pd.concat([grid_trip_num, grid_trip_num_hour], axis=0)

        
        print('网格内的trip总数量对比： real-generate{} vs {}'.format(real_trip_in_sihuan.shape[0] , trip_in_sihuan.shape[0] ) , end='----')
        grid_trip_num = grid_trip_num.fillna(0)
        # 多个poi列求和为一列 
        grid_trip_num['poi_num'] = grid_trip_num.iloc[:,4:22].sum(axis=1)
        # 保存
        grid_trip_num.to_csv('E:/Data/mobike201803/sample_rate_grid_with_hour/sample_rate{}_rect{}.csv'.format(samp_rate, grid_m) ,index=False,encoding='utf-8-sig')
        print('保存成功！')


  0%|          | 0/19 [00:00<?, ?it/s]

采样率： 2min  and   网格大小： 100 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 200 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 300 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 400 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 500 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 600 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 700 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 800 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 900 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 1000 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 1500 米----网格内的trip总数量对比： real-generate700545 vs 659093----保存成功！
采样率： 2min  and   网格大小： 2000 米----网格内的trip总数量对比： real

### 添加一些列

In [21]:
grid_trip_num

,poly_id,geometry,road_length,road_density,交通设施服务,住宿服务,体育休闲服务,公共设施,公司企业,医疗保健服务,商务住宅,地名地址信息,摩托车服务,政府机构及社会团体,汽车服务,汽车销售,生活服务,科教文化服务,购物服务,金融保险服务,风景名胜,餐饮服务,generate_trip_num,real_trip_num,hour
0,0,"POLYGON ((104.05436 30.56887, 104.05436 30.568...",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0
1,1,"POLYGON ((104.05436 30.56887, 104.05646 30.568...",0.083700,6.411440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0
2,2,"POLYGON ((104.05646 30.56887, 104.05855 30.568...",0.418245,13.808792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0
3,3,"POLYGON ((104.05855 30.56887, 104.06064 30.568...",0.603025,15.598243,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0
4,4,"POLYGON ((104.06064 30.56887, 104.06273 30.568...",0.512527,12.968720,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13935,13935,"POLYGON ((104.10246 30.78651, 104.10037 30.786...",0.202337,2.710032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,20
13936,13936,"POLYGON ((104.10456 30.78651, 104.10246 30.786...",0.500853,7.806123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,20
13937,13937,"POLYGON ((104.10665 30.78651, 104.10456 30.786...",0.420683,9.307686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,20
13938,13938,"POLYGON ((104.10874 30.78651, 104.10665 30.786...",0.159796,8.258599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,20


In [5]:
grid_m_list = np.append((np.linspace(100,900,9)),\
          np.linspace(1000,5000,9)) # 生成网格大小列表

for samp_rate in tqdm(range(2,21)):
    # 读取trip数据
    for grid_m in grid_m_list:
        grid_m = int(grid_m)
        print('采样率： %dmin  and'%samp_rate, '  网格大小：', grid_m, '米')
        samp_rate_grid = pd.read_csv('E:/Data/mobike201803/sample_rate_grid/sample_rate{}_rect{}.csv'.format(samp_rate,grid_m),encoding='utf-8-sig')
        from shapely import wkt
        samp_rate_grid['geometry'] = samp_rate_grid['geometry'].apply(wkt.loads)
        samp_rate_grid = gpd.GeoDataFrame(samp_rate_grid, geometry='geometry', crs='epsg:4326')
        samp_rate_grid = samp_rate_grid.to_crs('epsg:32648')
        samp_rate_grid['area'] = samp_rate_grid['geometry'].area / 1000000 # 平方公里

        # 添加列
        samp_rate_grid['samp_rate'] = samp_rate # 采样率
        samp_rate_grid['grid_m'] = grid_m # 网格大小
        # poi密度
        samp_rate_grid['poi_density'] = samp_rate_grid['poi_num'] / samp_rate_grid['area'] # poi密度
        # trip比例
        samp_rate_grid['trip_ratio'] = samp_rate_grid['real_trip_num'] / samp_rate_grid['generate_trip_num'] # trip比例
        # 转为wgs84坐标系
        samp_rate_grid = samp_rate_grid.to_crs('epsg:4326')
        # 保存
        samp_rate_grid.to_csv('E:/Data/mobike201803/sample_rate_grid/sample_rate{}_rect{}.csv'.format(samp_rate,grid_m),index=False,encoding='utf-8-sig')


  0%|          | 0/19 [00:00<?, ?it/s]

采样率： 2min  and   网格大小： 100 米
采样率： 2min  and   网格大小： 200 米
采样率： 2min  and   网格大小： 300 米
采样率： 2min  and   网格大小： 400 米
采样率： 2min  and   网格大小： 500 米
采样率： 2min  and   网格大小： 600 米
采样率： 2min  and   网格大小： 700 米
采样率： 2min  and   网格大小： 800 米
采样率： 2min  and   网格大小： 900 米
采样率： 2min  and   网格大小： 1000 米
采样率： 2min  and   网格大小： 1500 米
采样率： 2min  and   网格大小： 2000 米
采样率： 2min  and   网格大小： 2500 米
采样率： 2min  and   网格大小： 3000 米
采样率： 2min  and   网格大小： 3500 米
采样率： 2min  and   网格大小： 4000 米
采样率： 2min  and   网格大小： 4500 米
采样率： 2min  and   网格大小： 5000 米
采样率： 3min  and   网格大小： 100 米
采样率： 3min  and   网格大小： 200 米
采样率： 3min  and   网格大小： 300 米
采样率： 3min  and   网格大小： 400 米
采样率： 3min  and   网格大小： 500 米
采样率： 3min  and   网格大小： 600 米
采样率： 3min  and   网格大小： 700 米
采样率： 3min  and   网格大小： 800 米
采样率： 3min  and   网格大小： 900 米
采样率： 3min  and   网格大小： 1000 米
采样率： 3min  and   网格大小： 1500 米
采样率： 3min  and   网格大小： 2000 米
采样率： 3min  and   网格大小： 2500 米
采样率： 3min  and   网格大小： 3000 米
采样率： 3min  and   网格大小： 3500 米
采样率： 3min  and   网格大小： 4000 